In [ ]:
Tatenda

In [19]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

from pandas import Series, DataFrame
from pylab import rcParams
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report

In [20]:
metadata=pd.read_csv("movie_metadata_with_score_metacritic.csv", index_col="Unnamed: 0") #reading in metacritic dataset
metadata=metadata.loc[~metadata["metacritic_metascore"].isna()] #removing rows with no metacritic data
metadata=metadata.drop(["homepage","keywords","overview","status","tagline","imdb_metascore","budget","revenue"],1) #dropping unnecessary features
metadata=metadata.drop_duplicates() #dropping duplicates
metadata["release_year"]=metadata.release_date.str[0:4].astype(int) #creating new column with year for comparison

In [21]:
num_data=pd.read_csv("the_numbers_budget.csv") #reading in revenue data
num_data=num_data.rename(columns={"Movie":"title"}) #renaming column to match metadata
num_data["num_year"]=num_data["Release Date"].str[-4:].astype(int)
num_data=num_data.drop_duplicates()#dropping duplicates


In [22]:
critic_revenue=metadata.merge(num_data,on="title") #merging datasets
critic_revenue=critic_revenue.drop_duplicates() #dropping duplicates
critic_revenue=critic_revenue.loc[(critic_revenue["Worldwide Gross"]!=0)] #removing rows with no revenue data
critic_revenue=critic_revenue.loc[(np.abs(critic_revenue.release_year-critic_revenue.num_year)<5)] #removing rows where the years don't match, as this indicates different movies

critic_revenue["log Worldwide Gross"]=np.log(critic_revenue["Worldwide Gross"])#adding column for log of revenue

In [23]:
#Normalizing by the median for merged data set

critic_revenue["Production Budget"]=(critic_revenue["Production Budget"]-critic_revenue["Production Budget"].median())/critic_revenue["Production Budget"].std()
critic_revenue["Worldwide Gross"]=(critic_revenue["Worldwide Gross"]-critic_revenue["Worldwide Gross"].median())/critic_revenue["Worldwide Gross"].std()
critic_revenue["log Worldwide Gross"]=(critic_revenue["log Worldwide Gross"]-critic_revenue["log Worldwide Gross"].median())/critic_revenue["log Worldwide Gross"].std()
critic_revenue["Domestic Gross"]=(critic_revenue["Domestic Gross"]-critic_revenue["Domestic Gross"].median())/critic_revenue["Domestic Gross"].std()
critic_revenue["popularity"]=(critic_revenue["popularity"]-critic_revenue["popularity"].median())/critic_revenue["popularity"].std()
critic_revenue["vote_count"]=(critic_revenue["vote_count"]-critic_revenue["vote_count"].median())/critic_revenue["vote_count"].std()
critic_revenue["metacritic_metascore"]=(critic_revenue["metacritic_metascore"]-critic_revenue["metacritic_metascore"].median())/critic_revenue["metacritic_metascore"].std()
critic_revenue["runtime"]=(critic_revenue["runtime"]-critic_revenue["runtime"].median())/critic_revenue["runtime"].std()

# Reindex since the dataset has been subset many times
critic_revenue.index=range(len(critic_revenue))

In [25]:
#Replace every nan values with 0
critic_revenue.fillna(value=0,axis=1,inplace=True)
critic_revenue.shape

(3319, 21)

In [29]:
#Defining features and target for this dataset based on co-relation
features = ['Production Budget']
target = ['Worldwide Gross']

In [30]:
#splitting data set into training and test data set in 0.7/0.3
train, test = train_test_split(critic_revenue,test_size=0.30)
train.head()

,genres,id,original_language,original_title,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,...,vote_average,vote_count,metacritic_metascore,release_year,Release Date,Production Budget,Domestic Gross,Worldwide Gross,num_year,log Worldwide Gross
2295,"[{""id"": 35, ""name"": ""Comedy""}]",12479,en,Breakfast of Champions,-0.434055,"[{""name"": ""Hollywood Pictures"", ""id"": 915}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1999-02-13,0.243893,"[{""iso_639_1"": ""cs"", ""name"": ""\u010cesk\u00fd""...",...,5.2,-0.244632,-0.711091,1999,9/17/1999,-0.213462,-0.399895,-0.241760,1999,-2.423222
1265,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",72570,en,The Vow,0.297297,"[{""name"": ""Spyglass Entertainment"", ""id"": 158}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-02-05,-0.048779,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",...,7.0,0.707551,-0.656392,2012,2/10/2012,0.213462,1.440855,0.872933,2012,0.673722
1119,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...",943,en,Lethal Weapon 3,0.582465,"[{""name"": ""Silver Pictures"", ""id"": 1885}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1992-05-15,0.634123,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",...,6.4,0.344743,-0.820490,1992,5/15/1992,0.332052,1.731597,1.562174,1992,0.886222
2120,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 35, ...",114,en,Pretty Woman,0.864902,"[{""name"": ""Touchstone Pictures"", ""id"": 9195}, ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1990-03-23,0.682901,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",...,7.0,1.048530,-0.218797,1990,3/23/1990,-0.166026,2.228144,2.199836,1990,1.019866
85,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",68735,en,Warcraft,1.401611,"[{""name"": ""Universal Pictures"", ""id"": 33}, {""n...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}, {""iso...",2016-05-25,0.878016,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",...,6.3,1.441447,-1.258084,2016,6/10/2016,3.296804,0.293837,2.159758,2016,1.012558


In [31]:
#Fill the training and test data with require information
X_train = train[features].dropna()
y_train = train[target].dropna()
X_test = test[features].dropna()
y_test = test[target].dropna()

In [32]:
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor
model = ensemble.GradientBoostingRegressor()
model.fit(X_train, y_train)

/Users/tatenda/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [33]:
print('Gradient Boosting R squared": %.4f' % model.score(X_test, y_test))

Gradient Boosting R squared": 0.4453


In [34]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)
model_mse = mean_squared_error(y_pred, y_test)
model_rmse = np.sqrt(model_mse)
print('Gradient Boosting RMSE: %.4f' % model_rmse)

Gradient Boosting RMSE: 0.7141
